<a href="https://colab.research.google.com/github/nsharan2000/Underwater-image-classification/blob/main/Underwater_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Underwater image classification of multiple classes : 'auv': 0, 'fish': 1, 'jellyfish': 2, 'scubadiver': 3, 'shark': 4, 'starfish': 5, 'turtle': 6, 'wreckage': 7, with accuracy of 95 percentage.

Images scraped from internet and transfer learning is implemented for higher accuracy from a smaller dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import os

In [ ]:
import PIL

In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data_root = ('/content/drive/My Drive/dataset')

In [ ]:
IMAGE_SHAPE = (224, 224) # (height, width) in no. of pixels
img_height=224
img_width=224

In [ ]:
TRAINING_DATA_DIR = str(data_root)

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)

In [ ]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset='validation',
shuffle=True,
target_size=IMAGE_SHAPE
)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset='training',
shuffle=True,
target_size=IMAGE_SHAPE)

Found 543 images belonging to 8 classes.
Found 2193 images belonging to 8 classes.


In [ ]:
print (train_generator.class_indices)
class_names=list(train_generator.class_indices)

{'auv': 0, 'fish': 1, 'jellyfish': 2, 'scubadiver': 3, 'shark': 4, 'starfish': 5, 'turtle': 6, 'wreckage': 7}


In [ ]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
 f.write(labels)
!cat labels.txt

auv
fish
jellyfish
scubadiver
shark
starfish
turtle
wreckage

In [ ]:
import tensorflow_hub as hub

In [ ]:
model = tf.keras.Sequential([
 hub.KerasLayer('https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4', 
 output_shape=[1280],
 trainable=False),
 tf.keras.layers.Dropout(0.4),
 tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 8)                 10248     
Total params: 2,268,232
Trainable params: 10,248
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(
 optimizer=optimizer,
 loss='categorical_crossentropy',
 metrics=['accuracy'])

In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(
 train_generator, 
 epochs=100,
 verbose=1,
 steps_per_epoch=steps_per_epoch,
 validation_data=valid_generator,
 validation_steps=val_steps_per_epoch).history

Epoch 1/100
69/69 [==============================] - 677s 10s/step - loss: 1.0087 - accuracy: 0.6813 - val_loss: 0.3054 - val_accuracy: 0.9190
Epoch 2/100
69/69 [==============================] - 22s 316ms/step - loss: 0.2952 - accuracy: 0.9134 - val_loss: 0.2272 - val_accuracy: 0.9282
Epoch 3/100
69/69 [==============================] - 22s 321ms/step - loss: 0.2091 - accuracy: 0.9416 - val_loss: 0.2025 - val_accuracy: 0.9411
Epoch 4/100
69/69 [==============================] - 22s 325ms/step - loss: 0.1623 - accuracy: 0.9553 - val_loss: 0.1938 - val_accuracy: 0.9374
Epoch 5/100
69/69 [==============================] - 23s 338ms/step - loss: 0.1264 - accuracy: 0.9631 - val_loss: 0.1844 - val_accuracy: 0.9392
Epoch 6/100
69/69 [==============================] - 23s 339ms/step - loss: 0.1150 - accuracy: 0.9667 - val_loss: 0.1756 - val_accuracy: 0.9484
Epoch 7/100
69/69 [==============================] - 22s 315ms/step - loss: 0.0987 - accuracy: 0.9699 - val_loss: 0.1728 - val_accuracy: 

KeyboardInterrupt: ignored

In [ ]:
# The above code block is manually stopped to avoid overfitting of the model

In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

17/17 [==============================] - 6s 335ms/step - loss: 0.1827 - accuracy: 0.9448
Final loss: 0.18
Final accuracy: 94.48%


In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,50])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])


In [ ]:
plt.figure()
plt.ylabel(“Accuracy (training and validation)”)
plt.xlabel(“Training Steps”)
plt.ylim([0,1])
plt.plot(hist[“acc”])
plt.plot(hist[“val_acc”])

In [ ]:
model.save_weights('/content/drive/My Drive/model/model1')

In [ ]:
model.save('/content/drive/My Drive/model/fullmodel1')

INFO:tensorflow:Assets written to: /content/drive/My Drive/model/fullmodel1/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/model/fullmodel1/assets


In [ ]:
# path='/content/' + fn
# img=image.load_img(path, target_size=(150, 150))

# x=image.img_to_array(img)
# x=np.expand_dims(x, axis=0)
# images = np.vstack([x])

# classes = model.predict(images, batch_size=10)

tensorflow.python.keras.preprocessing.image.ImageDataGenerator

In [ ]:
# tf_model_predictions = model.predict(valid_generator)
# print("Prediction results shape:", tf_model_predictions.shape)
# plt.figure(figsize=(10,9))
# plt.subplots_adjust(hspace=0.5)
# for n in range((len(predicted_labels)-2)):
#  plt.subplot(6,5,n+1)
#  plt.imshow(val_image_batch[n])
#  color = "green" if predicted_ids[n] == true_label_ids[n] else "red"
#  plt.title(predicted_labels[n].title(), color=color)
#  plt.axis('off')
# _ = plt.subtitle('Model predictions (green: correct, red: incorrect)')

In [ ]:
predictions = model.predict(valid_generator)
print(predictions)
#this predictions is of the shape 548 * 8 as it gives the predicted values for all the 548 images in the test

[[4.45214791e-05 4.55672649e-04 1.06500365e-05 ... 3.89774141e-06
  9.99422431e-01 9.35303251e-06]
 [6.11826981e-05 3.25440167e-04 9.99379039e-01 ... 1.20098616e-06
  4.48658975e-05 4.26278857e-05]
 [4.22417978e-03 8.57818086e-05 1.42407557e-03 ... 1.75038203e-05
  7.65292571e-05 8.05195246e-04]
 ...
 [2.84813275e-03 7.19420239e-03 9.18048024e-01 ... 4.21545934e-03
  6.34440854e-02 2.51283543e-03]
 [5.77134604e-04 3.44763836e-03 6.75324816e-04 ... 6.58569916e-04
  3.31742398e-04 9.12853866e-05]
 [3.20725888e-03 1.27263227e-02 3.29258270e-04 ... 6.21251413e-04
  2.74588412e-04 5.75158454e-04]]


In [ ]:

# score = tf.nn.softmax(predictions[0])
# print(f"predictions={predictions}\n")
# print(f"This image most likely belongs to {class_names[np.argmax(score)]} with a {100 * np.max(score)} percent confidence.\n\n")

In [ ]:
# from sklearn.metrics import confusion_matrix

# predictions = model.predict(valid_generator)

# y_true = np.array([0] * 1000 + [1] * 1000)
# y_pred = predictions

# confusion_matrix(y_true, y_pred)

In [ ]:
# internetimage_url = "https://dtmag.com/wp-content/uploads/2006/08/greenseaturtle2-1050x700.jpg"
# image_path = tf.keras.utils.get_file('turtle1', origin=internetimage_url)

# img = tf.keras.preprocessing.image.load_img(
#     image_path, target_size=(img_height, img_width)
# )
# img_array = tf.keras.preprocessing.image.img_to_array(img)
# img_array = tf.expand_dims(img_array, 0) # Create a batch

# predictions = model.predict(img_array)
# score = tf.nn.softmax(predictions[0])
# print(f"predictions={predictions}\n")
# print(f"This image most likely belongs to {class_names[np.argmax(score)]} with a {100 * np.max(score)} percent confidence.\n\n")
# PIL.Image.open(image_path)
